In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (9001 Collins Ave s 205, Surfside, FL 33154, U...
2     (50 South Pointe Dr, Miami Beach, FL 33139, US...
3     (7821 Fisher Island Dr #7821, Miami Beach, FL ...
4     (2600 E Island Blvd PH 5, Aventura, FL 33160, ...
5     (1 Collins Ave Unit 304, Miami Beach, FL 33139...
6     (251 Crandon Blvd APT 937, Key Biscayne, FL 33...
7     (5325 Fisher Island Dr #5325, Miami Beach, FL ...
8     (9705 Collins Ave #1105n, Bal Harbour, FL 3315...
9     (15701 Collins Ave #1003, North Miami Beach, F...
10    (16001 Collins Ave APT 2801, Sunny Isles Beach...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Surf Club Residences\r9001 Collins Ave S-205\r...,10/30/2021,3/21/2022,101,"$14,000,000.00",3925,"$3,566.88",Elizabeth Rodriguez,Lazaro Rivero,Surf Club Residences 9001 Collins Ave S-205 Su...,Surf Club Residences,9001 Collins Ave S-205 Surfside,"(9001 Collins Ave s 205, Surfside, FL 33154, U...","(25.8768949, -80.12146349999999, 0.0)",25.876895,-80.121463,0.0
2,Continuum\r50 S Pointe Dr 1001/08\rMiami Beach,1/24/2022,3/24/2022,38,"$10,000,000.00",3497,"$2,859.59",Tracy Galya,Caryn Botknecht,Continuum 50 S Pointe Dr 1001/08 Miami Beach,Continuum,50 S Pointe Dr 1001/08 Miami Beach,"(50 South Pointe Dr, Miami Beach, FL 33139, US...","(25.767875, -80.132756, 0.0)",25.767875,-80.132756,0.0
3,Oceanside Fisher Island Condo\r7821 Fisher Isl...,10/15/2021,3/22/2022,126,"$8,900,000.00",3580,"$2,486.03",Jill Eber,Denisse Levy,Oceanside Fisher Island Condo 7821 Fisher Isla...,Oceanside Fisher Island Condo,7821 Fisher Island Dr 7821 Miami Beach,"(7821 Fisher Island Dr #7821, Miami Beach, FL ...","(25.75975, -80.13714499999999, 0.0)",25.759750,-80.137145,0.0
4,Residence Du Cap\r2600 Island Blvd PH-5\rAventura,1/31/2021,3/22/2022,381,"$5,400,000.00",5880,$918.37,Jonathan Laoui,Oscar Acosta,Residence Du Cap 2600 Island Blvd PH-5 Aventura,Residence Du Cap,2600 Island Blvd PH-5 Aventura,"(2600 E Island Blvd PH 5, Aventura, FL 33160, ...","(25.9390754, -80.13418039999999, 0.0)",25.939075,-80.134180,0.0
5,One Ocean Condo\r1 Collins Ave 304\rMiami Beach,12/10/2021,3/25/2022,77,"$3,725,000.00",2570,"$1,449.42",Constantin Gorges,Christopher Jude,One Ocean Condo 1 Collins Ave 304 Miami Beach,One Ocean Condo,1 Collins Ave 304 Miami Beach,"(1 Collins Ave Unit 304, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
6,Oceansound Key Colony\r251 Crandon Blvd 937\rK...,11/11/2021,3/25/2022,68,"$3,400,000.00",3143,"$1,081.77",Giulietta Ulloa,Wendy Mendoza PA,Oceansound Key Colony 251 Crandon Blvd 937 Key...,Oceansound Key Colony,251 Crandon Blvd 937 Key Biscayne,"(251 Crandon Blvd APT 937, Key Biscayne, FL 33...","(25.6977929, -80.1579812, 0.0)",25.697793,-80.157981,0.0
7,Bayview at Fisher Island\r5325 Fisher Island D...,12/3/2021,3/21/2022,53,"$3,295,000.00",2740,"$1,202.55",Jill Eber,Jonathan Kotler,Bayview at Fisher Island 5325 Fisher Island Dr...,Bayview at Fisher Island,5325 Fisher Island Dr 5325 Miami Beach,"(5325 Fisher Island Dr #5325, Miami Beach, FL ...","(25.763618, -80.148631, 0.0)",25.763618,-80.148631,0.0
8,St Regis Bal Harbour\r9705 Collins Ave 1105N\r...,8/9/2021,3/25/2022,186,"$2,900,000.00",1777,"$1,631.96",Sergio Balinotti,Diana Shay,St Regis Bal Harbour 9705 Collins Ave 1105N Ba...,St Regis Bal Harbour,9705 Collins Ave 1105N Bal Harbour,"(9705 Collins Ave #1105n, Bal Harbour, FL 3315...","(25.8887716, -80.1229685, 0.0)",25.888772,-80.122968,0.0
9,Ritz Carlton\r15701 Collins Ave 1003\rSunny Is...,2/25/2022,3/23/2022,12,"$2,700,000.00",1447,"$1,865.93",Lisa Miller PA,Ryan Mendell,Ritz Carlton 15701 Collins Ave 1003 Sunny Isle...,Ritz Carlton,15701 Collins Ave 1003 Sunny Isles Beach,"(15701 Collins Ave #1003, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
10,Trump Tower I\r16001 Collins Ave 2801\rSunny I...,12/13/2021,3/22/2022,60,"$2,579,070.00",2768,$931.74,Claudia Villar,Maria Kafetzis,Trump Tower I 16001 Collins Ave 2801 Sunny Isl...,Trump Tower I,16001 Collins Ave 2801 Sunny Isles Beach,"(16001 Collins Ave APT 2801, Sunny Isles Beach...","(25.9244562, -80.1216846, 0.0)",25.924456,-80.121685,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 20th - March 26th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')